In [18]:
import pandas as pd 
import networkx as nx
import dgl
import torch
from sklearn import model_selection
import numpy as np
import pickle
import h5py


Dataset = ['CPDB', 'IRefIndex', 'PCNet', 'IRefIndex_2015', 'STRINGdb', 'Multinet']

def get_health(dataset=0):
    k = pd.read_csv('/data/guest/GraphMAE/graphmae/datasets/health.tsv', sep='\t').astype(str)
    f = h5py.File('/data/guest/GraphMAE/graphmae/datasets/EMOGI_{}/{}_multiomics.h5'.format(dataset, dataset), 'r') 
    positive = k['symbol'].unique()
    name = [x.decode() for x in list(f['gene_names'][:][..., 1])]
    mask_test = f['mask_test'][:]
    mask_train = f['mask_train'][:]
    y_train = f['y_train'][:]
    positive_mask_test = np.nonzero(mask_test == True)[0]
    positive_mask_train = np.nonzero(mask_test == True)[0]
    neg_mask_train = []
    neg_mask_test = []
    for gene in positive[:60]:
        try: 
            i = name.index(gene)
            neg_mask_train.append(i)
        except ValueError:
            pass
    for gene in positive[60:]:
        try: 
            i = name.index(gene)
            neg_mask_test.append(i)
        except ValueError:
            pass
    pos_mask_test = positive_mask_test[:60]
    pos_mask_train = positive_mask_train[:120]
    t_mask = list(set(pos_mask_train) | set(neg_mask_train))
    train_mask = np.zeros_like(mask_train)
    test_mask = np.zeros_like(mask_test)
    te_mask = list(set(pos_mask_test) | set(neg_mask_test))
    train_mask[t_mask] = True
    test_mask[te_mask] = True
    print(len(t_mask), len(te_mask))
    y = np.zeros_like(y_train)
    for i in neg_mask_train:
        y[i] = True
    for i in neg_mask_test:
        y[i] = True
    src, dst = np.nonzero(f['network'][:])
    graph = dgl.graph((src, dst))
    graph.ndata['feat'] = torch.from_numpy(f['features'][:])   
    graph.ndata['train_mask'] = torch.from_numpy(train_mask)
    graph.ndata['val_mask'] = torch.from_numpy(test_mask)
    graph.ndata['test_mask'] = torch.from_numpy(test_mask)
    graph.ndata['label'] = torch.from_numpy(y).float()
    return graph, (graph.ndata["feat"].shape[1], 2)


def get_ppi(dataset=0, essential_gene=False, health_gene=False):
    dataset = Dataset[dataset]
    if health_gene:
        return get_health(dataset)
    elif essential_gene:
        f = h5py.File('/data/guest/GraphMAE/graphmae/essential_gene/{}_essential_test01_multiomics.h5'.format(dataset), 'r')
    else:
        f = h5py.File('/data/guest/GraphMAE/graphmae/datasets/EMOGI_{}/{}_multiomics.h5'.format(dataset, dataset), 'r') 
    src, dst = np.nonzero(f['network'][:])
    graph = dgl.graph((src, dst))
    #datas = torch.load("/data/guest/MTGCN/data/str_fearures.pkl").to("cpu")
    graph.ndata['feat'] = torch.from_numpy(f['features'][:])
    graph.ndata['train_mask'] = torch.from_numpy(f['mask_train'][:])
    graph.ndata['val_mask'] = torch.from_numpy(f['mask_test'][:])
    graph.ndata['test_mask'] = torch.from_numpy(f['mask_test'][:])
    graph.ndata['label'] = torch.from_numpy(np.logical_or(np.logical_or(f['y_test'][:], f['y_val'][:]), f['y_train'][:])).float()
    return graph, (graph.ndata["feat"].shape[1], 2)

x, _ = get_ppi()

64


In [15]:
x.subgraph(x.nodes()[x.ndata['train_mask']])

Graph(num_nodes=2013, num_edges=14330,
      ndata_schemes={'feat': Scheme(shape=(64,), dtype=torch.float64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(1,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})